In [1]:
#######################################################
##### STRIP TWEET #####################################
#######################################################

#import pandas as pd
#df = pd.read_csv("train2017.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
#corpus = df[3].tolist()

import pandas as pd
import re
import nltk
import string
df = pd.read_csv("small_train.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
train_corpus = df[3].tolist()
translate_table = dict((ord(char), None) for char in string.punctuation)   

clean_train_corpus = []
for tweet in train_corpus:
    tweet = re.sub(r"http\S+", "", tweet) #remove link
    tweet = ' '.join([word for word in tweet.split(' ')  if not word.startswith('@')])
    tweet = tweet.translate(translate_table) #remove symbols 
    tweet = tweet.lower()
    clean_train_corpus.append(tweet)

print(clean_train_corpus)
    

['gas by my house hit 339 im going to chapel hill on sat  😂', 'theo walcott is still shit fan watch rafa and johnny deal with him on saturday', 'its not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general says israels iron dome cant deal with their missiles keep talking like that and we may end up finding out', 'tehran mon amour obama tried to establish ties with the mullahs  via no barack obama  vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mashed out to niggas in paris in the club while in paris as cliche as it may sound weouthere', 'larry bird is ranked 4th alltime not including lebron or kobe just sayin']


In [2]:
#######################################################
##### TOKENIZATION ####################################
#######################################################

from nltk import word_tokenize

train_tokens = []
for tweet in clean_train_corpus:
    token = []
    token = word_tokenize(tweet)
    train_tokens.append(token)

print(train_tokens)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'going', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'is', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['its', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'says', 'israels', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missiles', 'keep', 'talking', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'finding', 'out'], ['tehran', 'mon', 'amour', 'obama', 'tried', 'to', 'establish', 'ties', 'with', 'the', 'mullahs', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mashed', 'out', 'to', 'niggas', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'as', 'cliche', 'as', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'is', 'ra

In [3]:
from nltk import pos_tag

from nltk.corpus import wordnet
from nltk.stem import  WordNetLemmatizer
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

train_tweets = []
for token_list in train_tokens:
    lemmatized = []
    for word in token_list:
        lemmatized.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    train_tweets.append(lemmatized)

print(train_tweets)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'be', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], ['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mash', 'out', 'to', 'nigga', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'a', 'cliche', 'a', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'be', 'ranked', '4th', 'alltime

In [4]:
final_train_corpus = []
for tweet in train_tweets:
    final_train_corpus.append(" ".join(str(word) for word in tweet))

print(final_train_corpus)

['gas by my house hit 339 im go to chapel hill on sat 😂', 'theo walcott be still shit fan watch rafa and johnny deal with him on saturday', 'it not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general say israel iron dome cant deal with their missile keep talk like that and we may end up find out', 'tehran mon amour obama try to establish tie with the mullah via no barack obama vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mash out to nigga in paris in the club while in paris a cliche a it may sound weouthere', 'larry bird be ranked 4th alltime not include lebron or kobe just sayin']


In [ ]:
#BAG-OF-WORDS VECTORIZATION
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_BOW = vectorizer.fit_transform(final_train_corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray()[0])
#print(X_BOW.toarray())


In [ ]:
#TF-IDF VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_TFIDF = vectorizer.fit_transform(final_train_corpus)
#print(X_TFIDF.toarray())


In [ ]:
#we have 3 options: word2vec, word2vec in sklearn, doc2vec
from gensim.test.utils import common_texts

#IMPORT WORD2VEC
from gensim.models import Word2Vec
nested_corpus = [final_train_corpus[i:i+1] for i in range(0, len(final_train_corpus), 1)]
model = Word2Vec(train_tokens, size=100, window=5, min_count=1, workers=4) #size of vector is 100
model.train(train_tokens, total_examples=model.corpus_count,epochs=model.epochs)  # train word vectors
print(model['gas'])

#IMPORT WORD2VEC LIBRARY AVAILABLE IN SKLEARN

#from gensim.sklearn_api import W2VTransformer
#total = sum(tokens,[])
#vectorizer = W2VTransformer(size=10, min_count=1, seed=1)
#print(corpus)
#X_embeddings = vectorizer.fit(total).transform(tokens)



[ 3.4717736e-03 -4.9537281e-03  2.8488671e-03 -4.5573632e-03
 -3.7937020e-03 -2.9213043e-04  3.6277922e-03 -1.5129922e-03
  9.7982993e-04 -8.8209321e-04  7.9758157e-04  2.5144566e-03
 -3.3176853e-03 -4.7022430e-03  2.0596345e-03 -2.7711764e-03
  8.3480478e-04  4.7617457e-03 -3.5368849e-03 -9.4520374e-06
 -2.9633900e-03  2.9271308e-03 -1.9926042e-03 -8.3342427e-04
  3.2666701e-04  1.2981131e-03 -4.3696197e-04  1.5445305e-03
 -1.7181310e-03 -5.0038104e-03 -1.5046336e-03 -3.1523132e-03
 -1.3387342e-03 -4.1482332e-03  2.0695240e-03 -1.6448352e-03
 -2.2167743e-03 -3.0591839e-03 -4.8770234e-03  4.9308681e-05
  1.8038582e-03 -6.5516349e-04 -3.5723748e-03  1.4162593e-03
  4.8562372e-03 -4.4696834e-03  2.5044356e-03  3.6430808e-03
  2.2215347e-03 -3.0143620e-03 -4.2602401e-03  8.2416000e-04
  1.8534381e-03  3.3984412e-05 -1.9368144e-03 -2.6199964e-03
  1.1428250e-03 -3.5262217e-03  1.4874688e-03 -2.0158708e-03
  2.6807149e-03 -3.1172248e-04  1.9078515e-03 -3.6214439e-03
  1.3176476e-03  4.20162

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [ ]:
#Doc2vec is equal with word2vec but is more appropriate for phrases (vectorize phrases instead of words). I.e 
#1.Manos leaves the office every day at 18:00 to catch his train
#2. This season is called Fall, because leaves fall from the trees.
#In this way we can capture the difference between the same word used in a different context. For example we now have a
#different representation of the word “leaves” in the above two sentences

#IMPORTANT: parameteres in my case is random, we have to pay attention to select the right ones

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(final_train_corpus)]
print(tagged_data)
model = Doc2Vec(size=200,
                alpha=model.alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

max_epoch = 20
for epoch in range(max_epoch):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
X_embeddings_array = model.docvecs
#convert array to list
X_embeddings=[]
for i in range(len(X_embeddings_array)):
    X_embeddings.append(X_embeddings_array[i].tolist())

/home/dimitra/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


[TaggedDocument(words=['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], tags=['0']), TaggedDocument(words=['theo', 'walcott', 'be', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], tags=['1']), TaggedDocument(words=['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], tags=['2']), TaggedDocument(words=['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], tags=['3']), TaggedDocument(words=['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], tags=['4']), TaggedDocument(words=['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], tags=

In [ ]:
#Add characteristics to embeddings
import pandas as pd
import csv
lexica =[pd.read_csv("lexica/affin/affin.txt", sep='\t', header=None),
              pd.read_csv("lexica/emotweet/valence_tweet.txt", sep='\t', header=None),
              pd.read_csv("lexica/generic/generic.txt", sep='\t', engine="python" ,quoting=csv.QUOTE_NONE,header=None),
              pd.read_csv("lexica/nrc/val.txt", sep='\t', engine="python",quoting=csv.QUOTE_NONE, header=None),
              pd.read_csv("lexica/nrctag/val.txt", sep='\t', header=None)]

#mean valence of each tweet
for tweet in range(len(train_tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(train_tokens[tweet])].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(train_tokens[tweet])][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:
             X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

#length of each tweet               
for tweet in range(len(train_tokens)):   #for each tweet
    X_embeddings[tweet].append(len(train_tokens[tweet]))


    
#min and max valence of each tweet
for tweet in range(len(train_tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(train_tokens[tweet])].empty: #search for tweets' tokens in lexicon
            max_valence = df[df[0].isin(train_tokens[tweet])][1].max() #get average sentiment for this tweet
            min_valence = df[df[0].isin(train_tokens[tweet])][1].min()
            X_embeddings[tweet].append(max_valence)
            X_embeddings[tweet].append(min_valence)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

    
#mean valence for each half of a tweet
for tweet in range(len(train_tokens)):   #for each tweet
    entire_tweet = tokens[tweet]
    first_half= entire_tweet[:len(entire_tweet)//2]
    second_half= entire_tweet[len(entire_tweet)//2:]
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(first_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(first_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        if not df[df[0].isin(second_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(second_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        
    
            
for i in range(len(X_embeddings)):
    print(len(X_embeddings[i]))

#sentiment_sum = 0
#words_found = 0
#for j in range(len(tokens[i])):
#    if not df[df[0] == tokens[i][j]].empty: #search for tweets' tokens in lexicon
#        sentiment_sum += df[df[0] == tokens[i][j]].iloc[0][1] #get sentiment
#        words_found++
        

        


In [ ]:
####################
#CONSTRUCT Y_LABELS#
####################

import pandas as pd
df = pd.read_csv("small.tsv", sep='\t', header=None)
y_labels = df[2].tolist() #sentiments


for n, value in enumerate(y_labels):
    if value == "positive":
        y_labels[n] = 2
    elif value =="negative":
        y_labels[n] = 0
    else:
        y_labels[n] =1

